# Models ensemble to achieve better test metrics

Models ensemble is a popular strategy in machine learning and deep learning areas to achieve more accurate and more stable outputs.  
A typical practice is:
* Split all the training dataset into K folds.
* Train K models with every K-1 folds data.
* Execute inference on the test data with all the K models.
* Compute the average values with weights or vote the most common value as the final result.
<p>
<img src="./images/models_ensemble.png" width="80%" alt='models_ensemble'>
</p>

MONAI provides `EnsembleEvaluator` and `MeanEnsemble`, `VoteEnsemble` post transforms.  
This tutorial shows how to leverage ensemble modules in MONAI to set up ensemble program.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/MONAI/blob/master/examples/notebooks/models_ensemble.ipynb)

## Setup environment

In [1]:
%pip install -qU "monai[ignite, nibabel]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
# temporarily need this, FIXME remove when 7143b8b released
%pip install -qU git+https://github.com/Project-MONAI/MONAI#egg=MONAI

Note: you may need to restart the kernel to use updated packages.


## Setup imports

In [3]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import glob
import logging
import os
import tempfile
import shutil
import sys

import IPython
import nibabel as nib
import numpy as np
import torch

from monai.config import print_config
from monai.data import CacheDataset, DataLoader, create_test_image_3d
from monai.engines import EnsembleEvaluator, SupervisedEvaluator, SupervisedTrainer
from monai.handlers import MeanDice, StatsHandler, ValidationHandler
from monai.inferers import SimpleInferer, SlidingWindowInferer
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from monai.transforms import (
    Activationsd,
    AsChannelFirstd,
    AsDiscreted,
    Compose,
    LoadNiftid,
    MeanEnsembled,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
    ToTensord,
    VoteEnsembled,
)
from monai.utils import set_determinism

print_config()

MONAI version: 0.2.0+74.g8e5a53e
Python version: 3.7.5 (default, Nov  7 2019, 10:50:52)  [GCC 8.3.0]
Numpy version: 1.19.1
Pytorch version: 1.6.0

Optional dependencies:
Pytorch Ignite version: 0.3.0
Nibabel version: 3.1.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: NOT INSTALLED or UNKNOWN VERSION.
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [4]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp if directory is None else directory
print(root_dir)

/home/bengorman/notebooks/


## Set determinism, logging, device

In [5]:
set_determinism(seed=0)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
device = torch.device("cuda:0")

## Generate random (image, label) pairs

Generate 60 pairs for the task, 50 for training and 10 for test.  
And then split the 50 pairs into 5 folds to train 5 separate models.

In [6]:
data_dir = os.path.join(root_dir, "runs")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    for i in range(60):
        im, seg = create_test_image_3d(128, 128, 128, num_seg_classes=1, channel_dim=-1)

        n = nib.Nifti1Image(im, np.eye(4))
        nib.save(n, os.path.join(data_dir, f"img{i}.nii.gz"))

        n = nib.Nifti1Image(seg, np.eye(4))
        nib.save(n, os.path.join(data_dir, f"seg{i}.nii.gz"))

images = sorted(glob.glob(os.path.join(data_dir, "img*.nii.gz")))
segs = sorted(glob.glob(os.path.join(data_dir, "seg*.nii.gz")))

train_files = list()
val_files = list()
for i in range(5):
    train_files.append(
        [
            {"image": img, "label": seg}
            for img, seg in zip(
                images[: (10 * i)] + images[(10 * (i + 1)) : 50],
                segs[: (10 * i)] + segs[(10 * (i + 1)) : 50],
            )
        ]
    )
    val_files.append(
        [
            {"image": img, "label": seg}
            for img, seg in zip(images[(10 * i) : (10 * (i + 1))], segs[(10 * i) : (10 * (i + 1))])
        ]
    )

test_files = [{"image": img, "label": seg} for img, seg in zip(images[50:60], segs[50:60])]

## Setup transforms for training and validation

In [7]:
train_transforms = Compose(
    [
        LoadNiftid(keys=["image", "label"]),
        AsChannelFirstd(keys=["image", "label"], channel_dim=-1),
        ScaleIntensityd(keys=["image", "label"]),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=[96, 96, 96],
            pos=1,
            neg=1,
            num_samples=4,
        ),
        RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
        ToTensord(keys=["image", "label"]),
    ]
)
val_transforms = Compose(
    [
        LoadNiftid(keys=["image", "label"]),
        AsChannelFirstd(keys=["image", "label"], channel_dim=-1),
        ScaleIntensityd(keys=["image", "label"]),
        ToTensord(keys=["image", "label"]),
    ]
)

## Define CacheDatasets and DataLoaders for train, validation and test

In [8]:
train_dss = [CacheDataset(data=train_files[i], transform=train_transforms) for i in range(5)]
train_loaders = [
    DataLoader(train_dss[i], batch_size=2, shuffle=True, num_workers=4) for i in range(5)
]

val_dss = [CacheDataset(data=val_files[i], transform=val_transforms) for i in range(5)]
val_loaders = [DataLoader(val_dss[i], batch_size=1, num_workers=4) for i in range(5)]

test_ds = CacheDataset(data=test_files, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=1, num_workers=4)

40/40 Load and cache transformed data:  [==============================]
40/40 Load and cache transformed data:  [==============================]
40/40 Load and cache transformed data:  [==============================]
40/40 Load and cache transformed data:  [==============================]
40/40 Load and cache transformed data:  [==============================]
10/10 Load and cache transformed data:  [==============================]
10/10 Load and cache transformed data:  [==============================]
10/10 Load and cache transformed data:  [==============================]
10/10 Load and cache transformed data:  [==============================]
10/10 Load and cache transformed data:  [==============================]
10/10 Load and cache transformed data:  [==============================]


## Define a training process based on workflows

More usage examples of MONAI workflows are available at: [workflow examples](https://github.com/Project-MONAI/MONAI/tree/master/examples/workflows).

In [9]:
def train(index):
    net = UNet(
        dimensions=3,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)
    loss = DiceLoss(sigmoid=True)
    opt = torch.optim.Adam(net.parameters(), 1e-3)

    val_post_transforms = Compose(
        [Activationsd(keys="pred", sigmoid=True), AsDiscreted(keys="pred", threshold_values=True)]
    )

    evaluator = SupervisedEvaluator(
        device=device,
        val_data_loader=val_loaders[index],
        network=net,
        inferer=SlidingWindowInferer(roi_size=(96, 96, 96), sw_batch_size=4, overlap=0.5),
        post_transform=val_post_transforms,
        key_val_metric={
            "val_mean_dice": MeanDice(
                include_background=True, output_transform=lambda x: (x["pred"], x["label"]),
            )
        },
    )
    train_handlers = [
        ValidationHandler(validator=evaluator, interval=4, epoch_level=True),
        StatsHandler(tag_name="train_loss", output_transform=lambda x: x["loss"]),
    ]

    trainer = SupervisedTrainer(
        device=device,
        max_epochs=4,
        train_data_loader=train_loaders[index],
        network=net,
        optimizer=opt,
        loss_function=loss,
        inferer=SimpleInferer(),
        amp=False,
        train_handlers=train_handlers,
    )
    trainer.run()
    return net

## Execute 5 training processes and get 5 models

In [10]:
models = [train(i) for i in range(5)]
IPython.display.clear_output()

## Define evaluation process based on `EnsembleEvaluator`

In [11]:
def ensemble_evaluate(post_transforms, models):
    evaluator = EnsembleEvaluator(
        device=device,
        val_data_loader=test_loader,
        pred_keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
        networks=models,
        inferer=SlidingWindowInferer(roi_size=(96, 96, 96), sw_batch_size=4, overlap=0.5),
        post_transform=post_transforms,
        key_val_metric={
            "test_mean_dice": MeanDice(
                include_background=True, output_transform=lambda x: (x["pred"], x["label"]),
            )
        },
    )
    evaluator.run()

## Evaluate the ensemble result with `MeanEnsemble`

`EnsembleEvaluator` accepts a list of models for inference and outputs a list of predictions for further operations.

Here the input data is a list or tuple of PyTorch Tensor with shape: [B, C, H, W, D].  
The list represents the output data from 5 models.  
And `MeanEnsemble` also can support to add `weights` for the input data:
* The `weights` will be added to input data from highest dimension.
* If the `weights` only has 1 dimension, it will be added to the `E` dimension of input data.
* If the `weights` has 3 dimensions, it will be added to `E`, `B` and `C` dimensions.  
For example, to ensemble 3 segmentation model outputs, every output has 4 channels(classes),  
The input data shape can be: [3, B, 4, H, W, D], and add different `weights` for different classes.  
So the `weights` shape can be: [3, 1, 4], like:  
`weights = [[[1, 2, 3, 4]], [[4, 3, 2, 1]], [[1, 1, 1, 1]]]`.

In [12]:
mean_post_transforms = Compose(
    [
        MeanEnsembled(
            keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
            output_key="pred",
            # in this particular example, we use validation metrics as weights
            weights=[0.95, 0.94, 0.95, 0.94, 0.90],
        ),
        Activationsd(keys="pred", sigmoid=True),
        AsDiscreted(keys="pred", threshold_values=True),
    ]
)
ensemble_evaluate(mean_post_transforms, models)

INFO:ignite.engine.engine.EnsembleEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs
INFO:ignite.engine.engine.EnsembleEvaluator:Epoch[1] Complete. Time taken: 00:00:02
INFO:ignite.engine.engine.EnsembleEvaluator:Got new best metric of test_mean_dice: 0.931878411769867
INFO:ignite.engine.engine.EnsembleEvaluator:Engine run complete. Time taken 00:00:02


## Evaluate the ensemble result with `VoteEnsemble`

Here the input data is a list or tuple of PyTorch Tensor with shape: [B, C, H, W, D].  
The list represents the output data from 5 models.

Note that:
* `VoteEnsemble` expects the input data is discrete values.
* Input data can be multiple channels data in One-Hot format or single channel data.
* It will vote to select the most common data between items.
* The output data has the same shape as every item of the input data.

In [13]:
vote_post_transforms = Compose(
    [
        Activationsd(keys=["pred0", "pred1", "pred2", "pred3", "pred4"], sigmoid=True),
        # transform data into discrete before voting
        AsDiscreted(keys=["pred0", "pred1", "pred2", "pred3", "pred4"], threshold_values=True),
        VoteEnsembled(keys=["pred0", "pred1", "pred2", "pred3", "pred4"], output_key="pred"),
    ]
)
ensemble_evaluate(vote_post_transforms, models)

INFO:ignite.engine.engine.EnsembleEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs
INFO:ignite.engine.engine.EnsembleEvaluator:Epoch[1] Complete. Time taken: 00:00:02
INFO:ignite.engine.engine.EnsembleEvaluator:Got new best metric of test_mean_dice: 0.9303768813610077
INFO:ignite.engine.engine.EnsembleEvaluator:Engine run complete. Time taken 00:00:02


## Cleanup data directory

Remove directory if a temporary was used.

In [14]:
if directory is None:
    shutil.rmtree(root_dir)